In [1]:
!pip install mxnet

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 46.9 MB 6.3 MB/s eta 0:00:01     |██████████████                  | 20.6 MB 4.6 MB/s eta 0:00:06     |█████████████████████████▎      | 37.0 MB 7.4 MB/s eta 0:00:02     |█████████████████████████▌      | 37.3 MB 7.4 MB/s eta 0:00:02
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.16
    Uninstalling graphviz-0.16:
      Successfully uninstalled graphviz-0.16
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# Vortrainiertes Modell

Hier verwenden wir ein bereits trainiertes, komplexes Convolutional Neural Network.

Das Netzwerk wurde auf dem CIFAR-10 Datensatz trainiert.

Es geht hier mehrheitlich darum, aufzuzeigen, wozu grosse CNNs bei **grossen Bilddatensätzen** in der Lage sind.

In [2]:
import numpy as np

from sklearn.metrics import accuracy_score

import pandas as pd
import pickle

In [3]:
!pip install tensorflow
!pip install gluoncv

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
# Class mapping from pretrained model
class_names = {
    0:'plane',
    1:'car',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}

## ResNet110 - Use pretrained model (on CIFAR 10)

Wir laden das bereits trainierte Netzwerk mittels der `mxnet` Library.

Wir müssen das Modell **nicht mehr trainieren** und laden es hier nur.

Das verwendete Modell ist eine Implementierung von diesem Paper: https://arxiv.org/abs/1603.05027

In [5]:
from mxnet.gluon.data.vision import transforms

import mxnet as mx
from gluoncv.model_zoo import get_model

ctx = [mx.cpu(0)]

# Get the model CIFAR_ResNet20_v1, with 10 output classes, without pre-trained weights
net = get_model('cifar_resnet110_v2', pretrained=True, classes=10)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])  # Normalization ist analog wie bei Training des Netwerkes
])

6326KB [00:01, 3479.49KB/s]                          


# Anwendung des Modells auf dem Test Set

Wenn wir mit der Leistung unseres Modells auf dem `Validation Set` zufrieden sind, können wir es auf das `Test Set` anwenden.

In [6]:
with open('../data/test.pkl', 'rb') as f:
    X_test = pickle.load(f)

In [7]:
%time

y_test_pred_prob = net(transform_test(mx.ndarray.array(X_test)))

y_test_pred = np.argmax(y_test_pred_prob.asnumpy(), axis=1)
# Ziffern zurück zu Labels transformieren (die Reihenfolge ist vom Netz vorgegeben)  
y_test_pred_enc = np.vectorize(class_names.get)(y_test_pred)

y_test_pred_df = pd.DataFrame(y_test_pred_enc, columns=['label'])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs


Um die Vorhersagen an Kaggle zu senden, schreiben wir sie in eine .csv-Datei, die Sie manuell übermitteln können.

In [8]:
y_test_pred_df.to_csv('../out/cnn_pretrained.csv', header=True, index_label='id')